In [1]:
import logging
import sys

import numpy as np
import matplotlib.pyplot as mplt

from apsuite.optimization.rcds import RCDS, RCDSParams

root = logging.getLogger()
root.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
root.addHandler(handler)


In [2]:
%matplotlib qt5

# Examples of usage

## Simple paraboloid minimization with RCDS 

Let's set up the optimzation for the following problem

$$ \mathrm{x}_{\text{min}} = \text{argmin}_{\mathrm{x}} f(\mathrm{x}) $$

where

$$f(\mathrm{x}) = \mathrm{x}^\intercal \mathrm{x} + \xi = \sum_i x_i ^2 + \xi$$ 

and $$\xi \sim \mathcal{N}(\mu = 0, \sigma = \hat{\sigma})$$. 

The objective function landscape is a paraboloid with a stocastic component simulating exprimental noise with standard deviaation $\hat{\sigma}$. Will assume $\hat{\sigma}=10^{-2}$, and focus on the two-dimensional problem, whith $\mathrm{x}\in[-3, -5] \times [4, 6]$

We'll create a class named `ParaboiloidOptimization`, which will be a child class from the `RCDS` class. The reason we do this is because the optimzation problem at sight might have additional parameters or specific routines.  For examples, take a look a the `injbo_optimize_rcds.py`, or `rcds_posang.py`, or `dynap_opt_rcds.py` scripts in the `apsuite/commisslib` submodule

### Create the optimization problem classes

We create the `Params` class and the optimization class

In [3]:
class ParaboloidOptimizationParams(RCDSParams):

    def __init__(self):
        super().__init__()

    # in this case the params class is basically the
    # default class inherited from RCDSParams
    # it contains only parameters relevant to the RCDS
    # algorithm itself

    # but maybe the optimzation expeiment depend on several
    # other parameters, such as a minimum reading from a given sensor
    # in order to proceed with optimzation, maybe the name of the
    # optimization variables & their upper/lower limits etc


In [4]:
class ParaboloidOptimization(RCDS):
    """Minimize a noisy paraboloid."""

    def __init__(self, use_thread=True, isonline=True):
        super().__init__(use_thread, isonline)
        self.params = ParaboloidOptimizationParams()


    def objective_function(self, pos):
        return np.dot(pos, pos) + self.params.noise_level * np.random.randn()


We instantiate the class object

In [5]:
opt = ParaboloidOptimization(use_thread=False, isonline=False)

and can check its default parameters

In [6]:
print(opt.params)

initial_stepsize              :      0.010 
noise_level                   :      0.000 
tolerance                     :      0.000 
orthogonality_threshold       :      0.900 
update_search_directions      : True       
linescan_num_pts              :          6 
linescan_min_pts_fit          :          4 
outlier_method                : Xiaobiao   
outlier_max_err_factor        :      3.000 
outlier_percentile_limit      :      0.250 
max_number_iters              :        100 
max_number_evals              :       1000 
boundary_policy               : ToNaN      

Positions           
limit_lower         
limit_upper         
initial_position    
Search Dir. 0       


nex we set the desired parameters

In [7]:
# problem-specific params
opt.params.limit_lower = [-3, -5]
opt.params.limit_upper = [4, 6]
# if the workflow of the objective function evaluations
# or other routines in the main optimzation class depend
# on additional parameters, they could be set here as well

# RCDS params
# these are the parameters RCDS requires for its internal
# logic and operations
opt.params.max_number_evals = 500
opt.params.max_number_iters = 5
opt.params.initial_stepsize = 0.0100
opt.params.noise_level = 1e-2
opt.params.tolerance = 1e-5

# initial position refers to the initial point in the
# optimaztion variables space
# you could set it to a particular value or sample
# the available search space ramdomly, as below
opt.params.initial_position = np.random.uniform(
    low=opt.params.limit_lower, high=opt.params.limit_upper)

# search directions along whith to start the line
# minimizations. It is a matrix with each row standing for a
# given direction
opt.params.initial_search_directions = np.eye(2)

In [8]:
print(opt.params)

initial_stepsize              :      0.010 
noise_level                   :      0.010 
tolerance                     :      0.000 
orthogonality_threshold       :      0.900 
update_search_directions      : True       
linescan_num_pts              :          6 
linescan_min_pts_fit          :          4 
outlier_method                : Xiaobiao   
outlier_max_err_factor        :      3.000 
outlier_percentile_limit      :      0.250 
max_number_iters              :          5 
max_number_evals              :        500 
boundary_policy               : ToNaN      

Positions               Dir. 0         Dir. 1     
limit_lower               -3             -5       
limit_upper                4              6       
initial_position        -0.411          -4.74     
Search Dir. 0              1              0       
Search Dir. 1              0              1       


**launch the optimzation run**

will stop if either the `max_number_iters` or the `max_number_evals` are reached, or even if the 
changes in the objective function are less than `tol`

In [9]:
opt.start()

Starting Optimization. Initial ObjFun: 22.6

Iteration 0001/0005
    Bracketing Dir. 1:  Delta = [-0.0571, 0.0924]   Objfun = 22.5
    Linescan in Dir. 1:    Delta = 0.00351   Objfun = 22.5
    Updated ObjFun largest decrease 0.164.

    Bracketing Dir. 2:  Delta = [-0.45, 0.1]   Objfun = 0.0537
    Linescan in Dir. 2:    Delta = -0.0189   Objfun = 0.00404
    Updated ObjFun largest decrease 22.455.

    Checking extension point.
    Dir. 2 not replaced:
        func0 >= func_e: False
        cond 2nd deriv.: True
End of iteration 0001: Final ObjFun = 0.00404

Iteration 0002/0005
    Bracketing Dir. 1:  Delta = [-0.0407, 0.0545]   Objfun = -0.00463
    Linescan in Dir. 1:    Delta = 0.00623   Objfun = 0.00448

    Bracketing Dir. 2:  Delta = [-0.0112, 0.0251]   Objfun = -0.00103
        # outliers removed = 1,   
    Linescan in Dir. 2:    Delta = 0.00602   Objfun = 0.00442
    Updated ObjFun largest decrease 0.000.

    Checking extension point.
    Replacing direction 2
        Brack

accompany the optimzation history

In [10]:
fig, ax = opt.plot_history()

In [11]:
fig, ax = opt.plot_knobspace_slice()

## Rosenbrock function minimzation with RCDS
a slightly more complicated objective function

- objective 
$$f(x_0, \dots, x_i) = \sum_{i=0}^N 100(x_{i+1} - x_i^2) + (1 - x_i)^2$$

For 2 dimensions, $N=2$, 
- global minimum at $x_g = (1, 1)$, with $f(x_g) = 0$

In [3]:
class RosenbrockOptimizationParams(RCDSParams):

    def __init__(self):
        super().__init__()

    # in this case the params class is basically the
    # default class inherited from RCDSParams
    # it contains only parameters relevant to the RCDS
    # algorithm itself

    # but maybe the optimzation expeiment depend on several
    # other parameters, such as a minimum reading from a given sensor
    # in order to proceed with optimzation, maybe the name of the
    # optimization variables & their upper/lower limits etc


In [4]:
class RosenbrockOptimization(RCDS):
    """Minimize a noisy paraboloid."""

    def __init__(self, use_thread=True, isonline=False):
        super().__init__(use_thread, isonline)
        self.params = RosenbrockOptimizationParams()


    def objective_function(self, pos):
        obj = 0
        for i in range(len(pos)-1):
            obj += 100 * (pos[i+1] - pos[i]**2)**2
            obj += (1 - pos[i])**2 + 1e-2 * np.random.randn()
        return obj


In [5]:
opt = RosenbrockOptimization(use_thread=False, isonline=False)

In [6]:
print(opt.params)

initial_stepsize              :      0.010 
noise_level                   :      0.000 
tolerance                     :      0.000 
orthogonality_threshold       :      0.900 
update_search_directions      : True       
linescan_num_pts              :          6 
linescan_min_pts_fit          :          4 
outlier_method                : Xiaobiao   
outlier_max_err_factor        :      3.000 
outlier_percentile_limit      :      0.250 
max_number_iters              :        100 
max_number_evals              :       1000 
boundary_policy               : ToNaN      

Positions           
limit_lower         
limit_upper         
initial_position    
Search Dir. 0       


In [7]:
# problem-specific params
opt.params.limit_lower = [-4, -4]
opt.params.limit_upper = [4, 4]

# RCDS params
opt.params.max_number_evals = 10000
opt.params.max_number_iters = 1000
opt.params.initial_stepsize = 0.0100
opt.params.noise_level = 1e-2
opt.params.tolerance = 1e-4


#opt.params.initial_position = np.random.uniform(
#    low=opt.params.limit_lower, high=opt.params.limit_upper)

opt.params.initial_position = [-1., -1.] # just a really bad starting point

opt.params.initial_search_directions = np.eye(2)

In [8]:
print(opt.params)

initial_stepsize              :      0.010 
noise_level                   :      0.010 
tolerance                     :      0.000 
orthogonality_threshold       :      0.900 
update_search_directions      : True       
linescan_num_pts              :          6 
linescan_min_pts_fit          :          4 
outlier_method                : Xiaobiao   
outlier_max_err_factor        :      3.000 
outlier_percentile_limit      :      0.250 
max_number_iters              :       1000 
max_number_evals              :      10000 
boundary_policy               : ToNaN      

Positions               Dir. 0         Dir. 1     
limit_lower               -4             -4       
limit_upper                4              4       
initial_position          -1             -1       
Search Dir. 0              1              0       
Search Dir. 1              0              1       


In [9]:
opt.start()

Starting Optimization. Initial ObjFun: 404

Iteration 0001/1000
    Bracketing Dir. 1:  Delta = [-0.15, 0.1]   Objfun = 108
    Linescan in Dir. 1:    Delta = -0.0258   Objfun = 90.5
    Updated ObjFun largest decrease 313.499.

    Bracketing Dir. 2:  Delta = [-0.15, 0.1]   Objfun = 4.88
    Linescan in Dir. 2:    Delta = -0.0245   Objfun = 1.02

    Checking extension point.
    Replacing direction 1
        Bracket result:  Delta = [-0.00833, 0.00833]   Objfun = 1.02
        Linescan result: Delta = -0.0245   Objfun = 1
End of iteration 0001: Final ObjFun = 1

Iteration 0002/1000
    Bracketing Dir. 1:  Delta = [-0.00694, 0.00694]   Objfun = 1
    Linescan in Dir. 1:    Delta = -0.00145   Objfun = 0.99
    Updated ObjFun largest decrease 0.012.

    Bracketing Dir. 2:  Delta = [-0.00694, 0.00694]   Objfun = 0.99
        # outliers removed = 1,   
    Linescan in Dir. 2:    Delta = 0.00193   Objfun = 0.978
    Updated ObjFun largest decrease 0.012.

    Checking extension point.
    

/home/matheus/repos/apsuite/apsuite/optimization/rcds.py:395: RuntimeWarning: invalid value encountered in divide
  new_dir = diff/_np.linalg.norm(diff)



    Bracketing Dir. 2:  Delta = [-5.05e-17, 2.93e-16]   Objfun = -0.0032
    Linescan in Dir. 2:    Delta = -3.09e-17   Objfun = 0.00758
    Updated ObjFun largest decrease 0.004.

    Checking extension point.
    Dir. 2 not replaced:
        func0 >= func_e: True
        cond 2nd deriv.: True
End of iteration 0207: Final ObjFun = 0.00758

Iteration 0208/1000
    Bracketing Dir. 1:  Delta = [-6.78e-19, 5.49e-19]   Objfun = -0.00804
    Linescan in Dir. 1:    Delta = -3.47e-20   Objfun = 0.00169
    Updated ObjFun largest decrease 0.006.

    Bracketing Dir. 2:  Delta = [-2.96e-11, 3.62e-09]   Objfun = -0.00286
        # outliers removed = 1,   
    Linescan in Dir. 2:    Delta = 5.02e-10   Objfun = -0.00177

    Checking extension point.
    Dir. 1 not replaced:
        func0 >= func_e: True
        cond 2nd deriv.: True
End of iteration 0208: Final ObjFun = -0.00177

Iteration 0209/1000
    Bracketing Dir. 1:  Delta = [-1.28e-14, 9.43e-12]   Objfun = -0.00276
    Linescan in Dir. 1:

In [14]:
fig, ax = opt.plot_history()

In [13]:
fig, ax = opt.plot_knobspace_slice()

In [12]:
positions_evaluated = np.array(opt.positions_evaluated)
positions_cumulated_optimum = opt.get_cumulated_optimum()[1]

fig, ax = mplt.subplots(figsize=(8,8))

Xgrid, Ygrid = np.meshgrid(np.linspace(-4, 4, 201), np.linspace(-4, 4, 201) )

Zgrid = np.zeros_like(Xgrid)
for i in range(Xgrid.shape[0]):
    for j in range(Xgrid.shape[1]):
        pos = [Xgrid[i, j], Ygrid[i, j]]
        Zgrid[i, j] = opt.objective_function(pos=pos)
Zgrid = np.log(Zgrid+1)

ax.pcolormesh(Xgrid, Ygrid, Zgrid)
ax.contour(Xgrid, Ygrid, Zgrid, levels=10, colors='black')
ax.set_xlabel('x0')
ax.set_ylabel('x1')


# Add all evaluations
ax.plot(positions_evaluated[:, 0], positions_evaluated[:, 1], 'o', color='white', alpha=0.4, mfc='none')

# ax.plot(positions_cumulated_optimum[:, 0],
#         positions_cumulated_optimum[:, 1],
#         color='red', alpha=0.4, marker='x')

for i in range(len(positions_cumulated_optimum) - 1):
    ax.annotate(
        '',
        xy=(positions_cumulated_optimum[i + 1, 0], positions_cumulated_optimum[i + 1, 1]),
        xytext=(positions_cumulated_optimum[i, 0], positions_cumulated_optimum[i, 1]),
        arrowprops=dict(arrowstyle="->", color='red', lw=1)
    )
ax.scatter(1, 1, 50, marker='o', color='orange', label="True minimum")
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

(-2.0, 2.0)

# General considerations for online experiments




- RCDS is most adequate for small to moderate parameter space dimensions (up to 20 variables)
- if objective is too noisy, consider averaging or taking the median
- RCDS is a single objective box-constrained optimzation algorithm
    - if, at any iteration, the algorithm proposes positions outside the variable limits indicated by the user, the positiosn are set to `nan`s and the objective function returns `nan`
    - more general constraints can be achieved with barrier-methods
    $$f(x) = \text{objective}(x) + \lambda ~ \text{constraint}(x)$$
    - eg.: unit circle in param-space
        $$\text{constraint(x)} = \text{max}(x_1^2 + x_2^2 - 1, 0)$$     
- multi-objective problems can be linearly combined into single objective with appropriate weights $w_i$
    $$f(x) = \sum_i w_i \text{objective}_i(x) $$

## Example of useage in real experiment with PVs

for more examples, check out 
- `lnls-fac/apsuite/commisslib/injbo_optimize_rcds.py`: optimization of injection effficiency at the Booster using positions & angles at the transport line, Booster ejection kicker strength, klystron 2 amplitude & phases and booster RF phase as optimization knobs 
- `lnls-fac/apsuite/commisslib/rcds_posang.py`: optimzation of injection efficiency at the storage ring using postions & angles, NLK kick strength as optimzation knobs (most simple example)
- `lnls-fac/apsuite/commisslib/opt_septa_feedforward.py`: optimzation of septa feedforward
- `lnls-fac/apsuite/commisslib/dynap_opt_rcds.py`: optimzation of storage ring dynamic aperture using sextuupole families as optimztion knobs 

In [3]:
class OnlineOptProblemParams(RCDSParams):

    def __init__(self):
        """."""
        super().__init__()
        self.problem_ppty1 = None
        self.problem_ppty2 = None
        self.problem_ppty3 = None


In [4]:
KNOBS = ["posx", "posy", "angx", "angy"]
LIMS_UPPER = [+5., +5., +2., +2.]
LIMS_LOWER = [-5., -5., -2., -2.]

In [ ]:
import time
from siriuspy.epics import PV


class OnlineOptProblem(RCDS):
    """."""
    def __init__(self, use_thread=True, isonline=True):
        """."""
        RCDS.__init__(use_thread, isonline)
        self.params = OnlineOptProblemParams()

        if self.isonline:
            self._create_connections()

    def set_positions(self, positions):
        """Set position (parameter space) to setpoint PVs.

        Args:
            positions (array, list): posx, posy, angx, angy
        """
        if positions is None:
            return

        posx = self.pvs["posx_sp"]
        posy = self.pvs["posy_sp"]
        angx = self.pvs["angx_sp"]
        angy = self.pvs["angy_sp"]
        posx.value, posy.value, angx.value, angy.value = positions
        self.wait_()


    def get_positions(self):
        """Get parameter-spcae position from PVs readback.

        Returns:
            list: with posx, posy, angx, angy
        """
        posx = self.pvs["posx_rb"].value
        posy = self.pvs["posy_rb"].value
        angx = self.pvs["angx_rb"].value
        angy = self.pvs["angy_rb"].value
        return [posx, posy, angx, angy]

    def objective_function(self, pos):
        """Optimization problem objective function.

        Args:
            pos (array, list): parameter space positions
            (posx, posy, angx, angy)

        Returns:
            float: objective function
        """
        self.set_positions(pos)
        obj = self.pvs["detector_measurment_rb"]
        # RCDS is a minimization algorithm.
        # for maximization, set obj = -obj
        return obj

    def _create_connections(self):
        pvname = ""
        self.pvs["posx_sp"] = PV(pvname)
        self.pvs["posy_sp"] = PV(pvname)
        self.pvs["angx_sp"] = PV(pvname)
        self.pvs["angy_sp"] = PV(pvname)
        self.pvs["posx_rb"] = PV(pvname)
        self.pvs["posy_rb"] = PV(pvname)
        self.pvs["angx_rb"] = PV(pvname)
        self.pvs["angy_rb"] = PV(pvname)

        self.pvs["detector_measurement_rb"] = PV(pvname)

    def wait_(self):
        """Wait PVs reach setpoint."""
        pv_response_time = 2  # [s]
        print('waiting pvs')
        time.sleep(pv_response_time)
        # alternatively, you can check if the
        # desired PVs readback or mon have reached the
        # desired values set up to a tolerance value

    def measure_objfunc_noise(self, nr_meas=5):
        """Evaluate objectuve function to estimate experimental noise.

        Args:
            nr_meas (int, optional): Nr. of measurements. Defaults to 5.
        """
        meas = []
        pos = self.get_positions()
        for _ in range(nr_meas):
            meas.append(self.objective_function(pos))
        mean, noise = np.mean(meas), np.std(meas)
        print(f"objective function mean:  {mean:.3f}")
        print(f"objective function noise: {noise:.3f}")
        self.data['measured_objfuncs_for_noise'] = meas
        self.data['measured_noise_level'] = noise
        return


**Instantiate optmization object.**
- `isonline=True` to create PVs or devices connections
- `use_thread=True` to launch cell in a different thread, allowing use of the notebook while optimzation runs (plotting history, checking data, processing data already taken, etc)

In [ ]:
opt = OnlineOptProblem(isonline=True, use_thread=True)

**Set parameters**

Noise measurement & estimation
- initially, set any value for the `noise_level`. 
- then, take measurements of the objective functin noise to get an estimate
- update `noise_level` with the measured value 

Initial postion
- you can choose to start the optimzation from the current positions, or from the average postion in param space or in the origin of it.

Initial search directions
- this is a matrix. Each row corresponds to a direction vector along which the minimum will be scanned during the fist RCDS iteration. After that, these direcetions may change, according to Poweel's method for updating search directions.

In [ ]:
# problem-specific params
opt.params.limit_lower = LIMS_LOWER
opt.params.limit_upper = LIMS_UPPER

# RCDS params
opt.params.max_number_evals = 500
opt.params.max_number_iters = 10
opt.params.initial_stepsize = 0.0100
opt.params.noise_level = 1e-3  # set initial guesse, then measure noise &
                               # update it here
opt.params.tolerance = 1e-4


opt.params.initial_position = np.random.uniform(
    low=opt.params.limit_lower,
    high=opt.params.limit_upper
)
# or
# current_pos = opt.get_positions()
# opt.params.initial_position = current_pos

param_space_dim = len(opt.params.limit_lower)
opt.params.initial_search_directions = np.eye(param_space_dim)

In [ ]:
opt.start()

In [ ]:
opt.plot_history()

In [ ]:
fname = "optimization_run1"
opt.save_data(fname=fname, overwrite=False)

In [ ]:
opt.load_and_apply(fname)

In [ ]:
opt.data.keys()